Project Code : Jayanth Appalla, Shobana Arumugamangalam

In [ ]:
import pandas as pd
from bs4 import BeautifulSoup
from textblob import TextBlob
import html
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from sklearn import metrics
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
def read_data(president): # read the data
  df = pd.read_excel(r'training-Obama-Romney-tweets.xlsx',president,header=None)
  return df

In [ ]:
def preprocess(df):
  df.drop(index = df.index[:2],axis = 0,inplace=True)
  df.rename(columns = {1:"date",2:"time",3:'Annotated_tweet',4:'Class'},inplace=True)
  df = df.loc[df['Class'].isin([-1,0,1,2])]
  df = df[['date','time','Annotated_tweet','Class']]
  df['Class'] = df['Class'].replace({2:0})
  df = df.reset_index() 
  return df

In [ ]:
def get_tweets(df): # For extracting the tweets
  tweets=[]
  for row in df.itertuples(index = True):
    tweets.append((getattr(row,'Annotated_tweet')))
  return tweets


In [ ]:
def clean_tweets(tweets):
  final_tweets=[]
  for i in range (len(tweets)):
    temp = str(tweets[i]).replace('\n','')
    temp1 = re.sub(re.compile('<.*?>'),'',temp)
    final_tweets.append(temp1)
  for i in range(len(final_tweets)):
    final_tweets[i] = re.sub(r'(@[A-Za-z0–9_]+)|[^\w\s]|#|http\S+','',final_tweets[i])
  return final_tweets 

In [ ]:
def clean_tweets2(final_tweets):
  sw = stopwords.words('english')
  sw.remove('not')
  stemmer= PorterStemmer()
  final_tweets1=[]
  final = []
  final1=[]
  final2=[]
  for i in range(len(final_tweets)):
    final_tweets1.append(word_tokenize(final_tweets[i]))

  for i in range(len(final_tweets1)):
    final_tweets1[i] = [word for word in final_tweets1[i] if not word in sw]

  for i in range(len(final_tweets1)):
    l = ''.join([str(elem) for elem in final_tweets[i]])
    final.append(l)
 
  for i in range(len(final)):
    temp = ''.join(c for c in final[i] if not c.isdigit())
    final1.append(temp)
  
  for i in range(len(final1)):
    stem_input=nltk.word_tokenize(final1[i].lower())
    stem_text=' '.join([stemmer.stem(word) for word in stem_input])
    final2.append(stem_text)

  return final2


In [ ]:
# data_obama = read_data('Obama')
# data_obama = preprocess(data_obama)
# tweets_obama = get_tweets(data_obama)
# tweets_obama = clean_tweets(tweets_obama)
# tweets_obama = clean_tweets2(tweets_obama)

data_romney = read_data('Obama')
data_romney = preprocess(data_romney)
tweets_romney = get_tweets(data_romney)
tweets_romney = clean_tweets(tweets_romney)
tweets_romney = clean_tweets2(tweets_romney)


In [ ]:
# newdata_obama = pd.DataFrame(list(zip(tweets_obama,list(data_obama['Class']))))
newdata_romney = pd.DataFrame(list(zip(tweets_romney,list(data_romney['Class']))))

In [ ]:
# newdata_obama.rename(columns={0:'tweet',1:'Class'},inplace=True)
newdata_romney.rename(columns={0:'tweet',1:'Class'},inplace=True)

In [ ]:
newdata_romney

,tweet,Class
0,kirkpatrick who wore a basebal cap embroid wit...,0
1,question if romney and obama had a childpunch ...,0
2,obama debat that cracker ass cracker tonight i...,1
3,rt slate blame obama for four death in libya b...,0
4,your miss the point im afraid you do not under...,0
...,...,...
6940,the reason ann romney and michel obama match l...,0
6941,obama kenakan cincin syahadat sejak sma,0
6942,bitch be like obamafood stamp lmao _ùâ,0
6943,presid barack obama and republican challeng mi...,0


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

def create_tfidfVectorizer(df):
  tfidf=TfidfVectorizer()
  
  tfidf_matrix=tfidf.fit_transform(df['tweet'])
  
  df_tfidf = pd.DataFrame(tfidf_matrix.todense())
  
  return df_tfidf

In [ ]:
target_df = newdata_romney['Class']
newdata_romney.drop('Class',axis=1,inplace=True)

In [ ]:
newdata_romney = create_tfidfVectorizer(newdata_romney)

In [ ]:
# from sklearn.model_selection import train_test_split
# x, x1, y, y1 = train_test_split(newdata_romney,target_df,test_size=0.3,random_state=2)


# Test Dataset

In [ ]:
testset = pd.read_excel('final-testData-no-label-Obama-tweets(2).xlsx',header=None)

In [ ]:
testset

,0,1
0,1,<e>Obama</e> has to maintain his professionali...
1,2,<e>Obama</e> went into the debate swinging and...
2,3,Ditto. I started @247LS 4 years ago. RT @bmorr...
3,4,I absolutely love <e>Obama</e>'s view in <a>im...
4,5,I'm agreeing completely with <e>Obama</e>'s st...
...,...,...
1946,1947,"@IngrahamAngle Of course O did say ""act of te..."
1947,1948,‰ÛÏIt is highly offensive that the <e>Obama</...
1948,1949,KEEP THE GOOD NEWS COMING! Obama's lead over ...
1949,1950,#Obama said <a>gas prices</a> low under Bush ...


In [ ]:
testset=testset[[1]]

In [ ]:
# testset.drop([0,1,2],axis=1,inplace=True)
testset 

,1
0,<e>Obama</e> has to maintain his professionali...
1,<e>Obama</e> went into the debate swinging and...
2,Ditto. I started @247LS 4 years ago. RT @bmorr...
3,I absolutely love <e>Obama</e>'s view in <a>im...
4,I'm agreeing completely with <e>Obama</e>'s st...
...,...
1946,"@IngrahamAngle Of course O did say ""act of te..."
1947,‰ÛÏIt is highly offensive that the <e>Obama</...
1948,KEEP THE GOOD NEWS COMING! Obama's lead over ...
1949,#Obama said <a>gas prices</a> low under Bush ...


In [ ]:
testset = testset.rename(columns = {1:'Annotated_tweet'})

In [ ]:
tweets_test = get_tweets(testset)
tweets_test = clean_tweets(tweets_test)
tweets_test = clean_tweets2(tweets_test)

In [ ]:
# tweets_test

In [ ]:
new_test = pd.DataFrame(tweets_test)
new_test.rename(columns={0:'tweet'},inplace=True)

In [ ]:
newdata_test = create_tfidfVectorizer(new_test)

# Modelling

In [ ]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
# model = RandomForestClassifier()
# newdata_romney = pd.DataFrame(SelectKBest(chi2, k=int(len(newdata_test.columns))).fit_transform(newdata_romney, target_df))
# model.fit(newdata_romney,target_df)
# pred_RFC = model.predict(newdata_test)


In [ ]:
Log_Reg = LogisticRegression(random_state=0,solver='lbfgs',max_iter=1000)
xtemp = pd.DataFrame(SelectKBest(chi2, k=int(len(newdata_test.columns))).fit_transform(newdata_romney,target_df))
Log_Reg.fit(xtemp,target_df)
prediction = Log_Reg.predict_proba(newdata_test)
pred_LR=[]
for i in range(len(prediction)):
  temp = list(prediction[i])
  val=temp.index(max(temp))-1
  pred_LR.append(val)

In [ ]:
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
import numpy as np

# print('accuracy of logistic regression model is :',accuracy_score(predictions,y1))

# print("accuracy of Random Forest Classifier : ",accuracy_score(predictions1,y1))

In [ ]:
from sklearn import svm

from matplotlib.colors import ListedColormap
from sklearn import metrics

# # clf1 is a linear svm classifier
# clf1 = svm.SVC(kernel = 'linear')
# xtemp = pd.DataFrame(SelectKBest(chi2, k=int(len(newdata_test.columns))).fit_transform(newdata_romney,target_df))
# clf1.fit(xtemp,target_df)
# pred_svm = clf1.predict(newdata_test)
# pred_svm
#Display the outcome of classification
# print(metrics.classification_report(y1, prediction))
# print("accuracy is : ",accuracy_score(prediction,y1))

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics


# clf = DecisionTreeClassifier(criterion = 'entropy')
# xtemp = pd.DataFrame(SelectKBest(chi2, k=int(len(newdata_test.columns))).fit_transform(newdata_romney,target_df))
# model = clf.fit(xtemp,target_df)

# pred_DT = clf1.predict(newdata_test)

# pred_DT

#Display the outcome of classification
# print(metrics.classification_report(y1, prediction))

# accuracy_score(y1,prediction)

In [ ]:
# from sklearn.naive_bayes import GaussianNB

# Gaussian_model=GaussianNB()
# newdata_romney = pd.DataFrame(SelectKBest(chi2, k=int(len(newdata_test.columns))).fit_transform(newdata_romney,target_df))
# Gaussian_model.fit(newdata_romney,target_df)
# pred_NB = Gaussian_model.predict(newdata_test)
# print("Accuracy",metrics.accuracy_score(, ypred))
# print("Confusion Matrix \n",metrics.confusion_matrix(y1, ypred))

In [ ]:
# from sklearn.linear_model import SGDClassifier
# sgd = SGDClassifier(loss='modified_huber', max_iter=1000, tol=1e-3,   n_iter_no_change=10, early_stopping=True, n_jobs=-1 )
# xtemp = pd.DataFrame(SelectKBest(chi2,k=int(len(newdata_test.columns))).fit_transform(newdata_romney,target_df))
# sgd.fit(xtemp,target_df)
# pred_SGD = sgd.predict(newdata_test)

In [ ]:
import numpy as np
from google.colab import files
l=[]
l.append('74 56')
for i in range(len(newdata_test)):
  l.append(str(i)+';;'+str(pred_LR[i]))
l1 = pd.DataFrame(l) 
np.savetxt(r'Obama', l1.values,fmt="%s")
files.download('Obama')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>